In [19]:
from decode_time_greedy import extract_routes, find_solution, repair_capacity, repair_distance

In [20]:
import random
def random_permutation(n):
    numbers = list(range(1, n+1))
    random.shuffle(numbers)
    return numbers

def random_binary_vector(n, p):
    vec = []
    for _ in range(n):
        if random.random() < p:   # random() ~ U[0,1)
            vec.append(0)
        else:
            vec.append(1)
    return vec

In [21]:
number_customer = 100
number_truck = 10
number_drone = 10
from data import load_data
problem = load_data(r"data\100customers\r110.txt", 100, 10, 15)
for customer in problem.customer_list:
    print(customer)

Customer(x=35, y=35, demand=0, time=0.0, service=0.0)
Customer(x=41, y=49, demand=10, time=1.625, service=0.008333333333333333)
Customer(x=35, y=17, demand=7, time=0.25, service=0.008333333333333333)
Customer(x=55, y=45, demand=13, time=1.325, service=0.008333333333333333)
Customer(x=55, y=20, demand=19, time=0.8875, service=0.008333333333333333)
Customer(x=15, y=30, demand=26, time=0.25, service=0.008333333333333333)
Customer(x=25, y=30, demand=3, time=0.675, service=0.008333333333333333)
Customer(x=20, y=50, demand=5, time=0.825, service=0.008333333333333333)
Customer(x=10, y=43, demand=9, time=0.7625, service=0.008333333333333333)
Customer(x=55, y=60, demand=16, time=0.6625, service=0.008333333333333333)
Customer(x=30, y=60, demand=16, time=1.2625, service=0.008333333333333333)
Customer(x=20, y=65, demand=12, time=0.4125, service=0.008333333333333333)
Customer(x=50, y=35, demand=19, time=0.475, service=0.008333333333333333)
Customer(x=30, y=25, demand=23, time=0.875, service=0.00833

In [22]:
per_customers = random_permutation(number_customer + number_truck + number_drone -2)
drone_truck_assign = random_binary_vector(number_customer + number_truck + number_drone -2, 0.5)
chromosome = [per_customers, drone_truck_assign]
chromosome = repair_distance(chromosome, problem)
chromosome = repair_capacity(chromosome, problem)
print(chromosome)

[[68, 101, 82, 66, 102, 65, 41, 103, 19, 104, 17, 86, 105, 40, 51, 97, 2, 78, 80, 81, 12, 49, 13, 39, 106, 55, 43, 67, 32, 74, 85, 70, 84, 44, 42, 16, 107, 48, 108, 93, 56, 94, 109, 3, 28, 57, 29, 61, 7, 38, 11, 79, 25, 26, 90, 22, 52, 92, 27, 62, 77, 54, 71, 83, 1, 110, 64, 34, 23, 5, 9, 111, 63, 4, 100, 35, 72, 50, 47, 21, 75, 59, 53, 99, 112, 37, 20, 31, 91, 30, 6, 24, 95, 45, 113, 114, 98, 73, 8, 96, 115, 15, 33, 87, 18, 116, 76, 10, 117, 60, 46, 89, 14, 118, 69, 36, 88, 58], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


In [23]:
# Chạy thuật toán và in kết quả

assigned_truck_customers, assigned_drone_customers = extract_routes(chromosome, problem)
truck_solutions, drone_solutions = find_solution(assigned_truck_customers, assigned_drone_customers, problem)
while truck_solutions == False:
    idx = chromosome[0].index(drone_solutions)
    chromosome[1][idx] = 0
    chromosome = repair_distance(chromosome, problem)
    chromosome = repair_capacity(chromosome, problem)
    assigned_truck_customers, assigned_drone_customers = extract_routes(chromosome, problem)
    truck_solutions, drone_solutions = find_solution(assigned_truck_customers, assigned_drone_customers, problem)

    
# In kết quả cho các truck
print("Kết quả cho Truck:")
for truck_solution in truck_solutions:
    print(f"  - Khách hàng được giao: {truck_solution.assigned_customers}")
    print(f"  - Thời gian đến: {truck_solution.arrive_time}")
    print(f"  - Thời gian rời đi: {truck_solution.leave_time}")
    print(f"  - Lượng hàng truck nhận: {truck_solution.recived_truck}")
    print(f"  - Lượng hàng drone nhận: {truck_solution.recived_drone}")
    print("-" * 20)

# In kết quả cho các drone
print("\nKết quả cho Drone:")
for i, drone_solution in enumerate(drone_solutions):
    print(f"  - Trip {i+1}:")
    if not drone_solution.trip_list:
        print("    - Không có chuyến bay khả thi.")
        continue

    for j, sortie in enumerate(drone_solution.trip_list):
        print(sortie.assigned_customers)
        print(sortie.arrive_time)
        print(sortie.leave_time)
        print(sortie.recived_drone)
    print("=" * 30)

Kết quả cho Truck:
  - Khách hàng được giao: [68, 79]
  - Thời gian đến: [0.625      1.28333333]
  - Thời gian rời đi: [1.03333333 1.29166667]
  - Lượng hàng truck nhận: [36. 23.]
  - Lượng hàng drone nhận: [0. 0.]
--------------------
  - Khách hàng được giao: [82, 66]
  - Thời gian đến: [0.8        2.30833333]
  - Thời gian rời đi: [0.80833333 2.31666667]
  - Lượng hàng truck nhận: [16. 25.]
  - Lượng hàng drone nhận: [0. 0.]
--------------------
  - Khách hàng được giao: [65, 41, 47]
  - Thời gian đến: [1.725      3.98333333 6.06666667]
  - Thời gian rời đi: [1.73333333 3.99166667 6.075     ]
  - Lượng hàng truck nhận: [20.  5. 27.]
  - Lượng hàng drone nhận: [0. 0. 0.]
--------------------
  - Khách hàng được giao: [19, 23]
  - Thời gian đến: [1.125      3.50833333]
  - Thời gian rời đi: [1.13333333 3.51666667]
  - Lượng hàng truck nhận: [17. 29.]
  - Lượng hàng drone nhận: [0. 0.]
--------------------
  - Khách hàng được giao: [17, 86]
  - Thời gian đến: [0.875      1.20833333]
  

In [24]:
for truck_sol in truck_solutions:
    print(problem.check_capacity_truck_contraint(truck_sol))


print("+++++++++++++++++++++++")
for drone_sol in drone_solutions:
    print(problem.check_capacity_drone_constraint(drone_sol))
    for drone_trip in drone_sol.trip_list:
        print("Check energy")
        print(problem.check_energy_drone(drone_trip))
        print(drone_trip.assigned_customers)
        print(drone_trip.recived_drone)
        print(drone_trip.arrive_time)
        print([problem.customer_list[x].arrive_time for x in drone_trip.assigned_customers])
        print(drone_trip.leave_time)

    print("++++++++++++++++++++++++++++++++++")

True
True
True
True
True
False
True
True
True
True
+++++++++++++++++++++++
True
Check energy
i =  0
wait_time:  0
i =  1
wait_time 2: 0
total_energy 68.53506932918697 180
True
[0, 83, 0]
[0, 11, 0]
[0, 0.38491705182881647, 0.38491705182881647]
[0.0, 0.2625, 0.0]
[0, 0.39325038516214983, 0.38491705182881647]
Check energy
i =  0
wait_time:  0
i =  1
wait_time 2: 0
total_energy 81.73529270389177 180
True
[0, 62, 0]
[0, 19, 0]
[0.38491705182881647, 0.8431686779615486, 0.8431686779615486]
[0.0, 0.3125, 0.0]
[0.38491705182881647, 0.8515020112948819, 0.8431686779615486]
Check energy
i =  0
wait_time:  0
i =  1
wait_time 2: 0
total_energy 124.23319209031165 180
True
[0, 71, 0]
[0, 15, 0]
[0.8431686779615486, 1.5375197451299467, 1.5375197451299467]
[0.0, 0.7125, 0.0]
[0.8431686779615486, 1.54585307846328, 1.5375197451299467]
Check energy
i =  0
wait_time:  0
i =  1
wait_time 2: 0
total_energy 64.34314681077663 180
True
[0, 77, 0]
[0, 14, 0]
[1.5375197451299467, 1.8991483385231502, 1.89914833852